In [4]:
import pandas as pd

stops = pd.read_csv('gtfs_data/stops.csv')
# Load translations from csv
translations = pd.read_csv('gtfs_data/translations.csv')
stop_times = pd.read_csv('gtfs_data/stop_times.csv')
routes = pd.read_csv('routes_with_data.csv')

# Merge stops DataFrame with translations DataFrame on stop_id
stops_with_translations = stops.merge(translations, left_on='stop_name', right_on='field_value', how='left')
stops_with_translations['stop_name'] = stops_with_translations['translation']

stops_with_translations = stops_with_translations[stops_with_translations['language'] == 'nl']

# Remove rows with non-numeric stop_id
stops_with_translations = stops_with_translations[stops_with_translations['stop_id'].astype(str).str.isdigit()]
stop_times = stop_times[stop_times['stop_id'].astype(str).str.isdigit()]

stops_with_translations['stop_id'] = stops_with_translations['stop_id'].astype(str)
stop_times['stop_id'] = stop_times['stop_id'].astype(str)

stops_temp = stops_with_translations.merge(stop_times, on='stop_id', how='left')
stops_trip = stops_temp.merge(routes, on='trip_id', how='left')

# stops_trip = stops_with_translations.merge(stop_times, on='stop_id', how='left')
# stops_trip['from'] = stops_trip['trip_id'].str.split(':').str[3]
# stops_trip['to'] = stops_trip['trip_id'].str.split(':').str[4]

stops_no_dup = stops_trip[['stop_id', 'stop_name', 'route_id', 'stop_sequence', 'trip_id']].drop_duplicates()
stops_no_dups = stops_no_dup.dropna(subset=['route_id'], inplace=False).drop_duplicates()
# Create a CLINGO file with unique stops predicate
with open("stops.lp", "w") as f:
    for index, row in stops_no_dup.iterrows():
        if pd.isna(row['stop_name']):
            continue  # Skip rows where no translation was found
        if({str(row['route_id']).split('.')[0]}):
            f.write(f'''stops("{row['trip_id']}", {row['stop_id']}, "{row['stop_name'].lower()}", {row['stop_sequence']}).\n''')

In [ ]:
stops_with_translations.head()

In [ ]:
stops_no_dup

In [ ]:
stops_no_dup = stops_no_dup.dropna(subset=['route_id'], inplace=False).drop_duplicates()

In [ ]:
stops_no_dup.head()

In [ ]:
stops_no_dup.info()

In [ ]:
import networkx as nx
from networkx import DiGraph
import plotly.graph_objects as go

# Function to create a graph for each route
def create_graph_for_route(route_data):
    G = DiGraph()
    for _, row in route_data.iterrows():
        stop_id = row['stop_id']
        place = row['stop_sequence']
        route = str(row['route_id']).split('.')[0]
        if not G.has_node(stop_id):
            G.add_node(stop_id)
        G.nodes[stop_id]['label'] = row['stop_name']
        if place > 1:
            prev_stop_id = route_data.loc[route_data['stop_sequence'] == place - 1, 'stop_id'].values
            if len(prev_stop_id) > 0:
                G.add_edge(prev_stop_id[0], stop_id)
    return G

# Check if stops_no_dup is empty
if stops_no_dup.empty:
    print("stops_no_dup is empty")
else:
    # Group the DataFrame by route_id and create a graph for each route
    graphs = {}
    for route_id, route_data in stops_no_dup.groupby('route_id'):
        if len(route_data) == 0:
            print(f"Group {route_id} is empty")
        else:
            graphs[route_id] = create_graph_for_route(route_data)

    # Combine all graphs into one large graph using nx.compose_all
    large_graph = nx.compose_all(graphs.values())

# Convert the NetworkX graph to a Plotly graph
pos = nx.kamada_kawai_layout(large_graph)  # positions for all nodes

edge_x = []
edge_y = []

for edge in large_graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []

# Assuming 'stop_id' is the label you want to use
for node in large_graph.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    text=[f"{str(large_graph.nodes[node]['label']).capitalize()}<br>{large_graph.degree(node)} connections" for node in large_graph.nodes()],
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        line_width=2))

# Create a color list for edges
edge_colors = ['black'] * len(large_graph.edges())

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                 title='<br>Network graph',
                 titlefont_size=16,
                 showlegend=False,
                 hovermode='closest',
                 margin=dict(b=20,l=5,r=5,t=40),
                 xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                 yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

# Node ID you want to autozoom into
target_node = '8891702'

if target_node in pos:
    # Get the coordinates of the target node
    x_target, y_target = pos[target_node]
    
    # Create a figure with the edges and nodes
    fig = go.Figure(data=[edge_trace, node_trace])
    
    # Update layout to center on the target node and adjust zoom level
    fig.update_layout(
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        title_text=f'Network graph autozoomed into Node {target_node}',
        scene=dict(
            camera=dict(eye=dict(x=-1.25, y=-1.25, z=0.7)),
            xaxis_title='X-axis',
            yaxis_title='Y-axis',
            zaxis_title='Z-axis'
        )
    )

    # Adjust the layout to center on the target node
    fig.update_layout(
        scene=dict(
            camera=dict(eye=dict(x=-1.25, y=-1.25, z=0.7)),
            xaxis=dict(range=[x_target - 1, x_target + 1]),
            yaxis=dict(range=[y_target - 1, y_target + 1])
        )
    )

else:
    print(f"Target node '{target_node}' not found in the graph.")

fig.show()

In [ ]:
edge_x

In [ ]:
import plotly.graph_objs as go


# Initialize lists to store the CLingo facts
clingo_facts = []

# Convert nodes to CLingo facts
for node_id in pos:
    clingo_facts.append(f'node({node_id}).')

# Convert edges to CLingo facts
edge_count = min(len(edge_x), len(edge_y))
for i in range(edge_count):
    source_node = str(node_x[i])
    target_node = str(node_y[i])
    if source_node not in pos or target_node not in pos:
        print(f"Warning: Source node {source_node} or target node {target_node} is not a valid node.")
        continue
    clingo_facts.append(f'edge({source_node}, {target_node}).')

# Print the CLingo facts
for fact in clingo_facts:
    print(fact)

# Create the Plotly graph
edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Viridis',
        size=10,
        colorbar=dict(title='Node ID'),
        line_width=2
    )
)

# Create a figure and add the traces
fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20,l=5,r=5,t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# Plot the graph
fig.show()

# Example of checking node existence in CLingo facts
def check_node_exists(node_id):
    return any(fact.startswith(f'node({node_id})') for fact in clingo_facts)

print(check_node_exists('1'))  # Should return True
print(check_node_exists('5'))  # Should return False

In [ ]:
import unicodedata

def normalize_string(s):
    # Normalize the string to NFKD form and encode it to ASCII bytes, ignoring errors
    normalized = unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('ascii')
    return normalized

# Example usage
string = "è, é, ê, and other accents"
normalized_string = normalize_string(string)
print(normalized_string)  # Output: e, e, e, and other accents

In [ ]:
def export_to_asp(graph, file_path):
    with open(file_path, 'w') as file:
        for node, data in graph.nodes(data=True):
            line_numbers = data.get('line_numbers', [])
            for line in line_numbers:
                file.write(f"node({normalize_string(node).replace('-', '_').replace(' ', '_')}, {int(extract_integers_from_string(line)[0])}).\n")
                #print(f"node({normalize_string(node).replace('-', '_').replace(' ', '_')}, {int(extract_integers_from_string(line)[0])}).\n")
        for node1, node2, data in graph.edges(data=True):
            file.write(f"edge({normalize_string(graph.nodes[node1]['label']).replace('-', '_').replace(' ', '_').lower()}, {normalize_string(graph.nodes[node2]['label']).replace('-', '_').replace(' ', '_').lower()}).\n")

# Export the combined graph to a .lp file
export_to_asp(large_graph, 'graph.lp')